In [1]:
import tracers as tr
import numpy as np
import plotly.graph_objects as go
import datetime

In [2]:
mypygraph = tr.PyGraph()
print(mypygraph.yendepth)

mypygraph.load_graph("/home/toon/Documents/PHD/coding/osrm-docker/data-volume/monaco/monaco-latest.osm.pbf")

2


In [6]:
mypygraph.map_match_multiple_traces([[
        (7.4132824, 43.7274446,0),
        (7.4149668, 43.7284912,1000),
        (7.4164581, 43.7297472,2000),
        (7.4188507, 43.7311969,3000),
        (7.4214149, 43.7318249,4000),
        (7.4230134, 43.7321117,5000),
        (7.4216938, 43.7336234,6000),
        (7.4212968, 43.7361196,7000),
        (7.4257171, 43.7376855,8000),
        (7.4252021, 43.7397164,9000),
        (7.4232495, 43.7397242,10000),
        (7.4217904, 43.7398792,11000),
    ]], 50.0, "./testout/")

In [ ]:
infile = r"/home/toon/Documents/PHD/coding/pyrust-docker/tracers/tests/testout/_0.csv"

#even lines are the matched points
#odd lines are the original input

matches = []
ogpoints = []

with open(infile, "r") as f:
    for matchline in f:
        matchline = matchline.rstrip()  # remove newline character
        if matchline:
            # process matchline, it has format int,float,float,int|...
            strtupls = matchline.split("|")
            matches.append([(int(split[0]),float(split[1]),float(split[2]),int(split[3]))for split in [x.split(",") for x in strtupls]])

        # origline
        origline = f.readline()
        #strip
        origline = origline.rstrip()
        if origline:
            # process origline, it has format float,float,int|...
            split = origline.split(",")
            ogpoints.append((float(split[0]),float(split[1]),int(split[2])))




In [ ]:
ogpoints

In [ ]:
[x for x in ogpoints]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scattermapbox(
    lat=[x[1] for x in ogpoints],
    lon=[x[0] for x in ogpoints],
    mode='markers',
    marker=dict(size=9),
    line=dict(width=2),
    name="Original"
))
for matchtrace in matches:
    fig.add_trace(go.Scattermapbox(
        lon=[x[1] for x in matchtrace],
        lat=[x[2] for x in matchtrace],
        mode='lines',
        marker=dict(size=9),
        line=dict(width=2),
        name="Matched",
        line_color="red"
    ))
fig.update_layout(
    mapbox_style="open-street-map",
    mapbox=dict(
        center=dict(
            lat=43.7274446,
            lon=7.4132824
        ),
        zoom=14
    )
)

In [ ]:
# # write result[0] to pickle file
# import pickle
# with open('output/result.pickle', 'wb') as f:
#     pickle.dump(result[0], f)


In [ ]:
simple_lonlat_list = [
    (7.4132824, 43.7274446),
    (7.4149668, 43.7284912),
    (7.4164581, 43.7297472),
    (7.4188507, 43.7311969),
    (7.4214149, 43.7318249),
    (7.4230134, 43.7321117),
    (7.4216938, 43.7336234),
    (7.4212968, 43.7361196),
    (7.4257171, 43.7376855),
    (7.4252021, 43.7397164),
    (7.4232495, 43.7397242),
    (7.4217904, 43.7398792)
]

per_simple_lonlat = tr.uniform_perturb_trace(simple_lonlat_list, 20.0)


In [ ]:
mypygraph.yendepth = 5

In [ ]:
# (mapmatch_result, mapmatch_microresult) =  mypygraph.map_match_trace(per_simple_lonlat, 200.0)
(mapmatch_result, mapmatch_microresult) =  mypygraph.map_match_trace(simple_lonlat_list, 50.0)

In [ ]:
matched_lonlat = [(lon,lat) for (_,lon,lat) in map(mypygraph.get_node_weight, mapmatch_result)]
matched_micro_lonlat = [(lon,lat) for (_,lon,lat) in map(mypygraph.get_node_weight, mapmatch_microresult)]

# plot the matched_lonlat on the map with plotly mapbox
fig = go.Figure(go.Scattermapbox(
        lat=[lat for (_,lat) in matched_lonlat],
        lon=[lon for (lon,_) in matched_lonlat],
        mode='lines',
        marker=go.scattermapbox.Marker(
            size=9
        ),
        text=[f'lon: {lon}, lat: {lat}' for (lon,lat) in matched_lonlat]
    ))

fig.add_trace(go.Scattermapbox(
        lat=[lat for (_,lat) in per_simple_lonlat],
        lon=[lon for (lon,_) in per_simple_lonlat],
        mode='lines',
        marker=go.scattermapbox.Marker(
            size=9
        ),
        text=[f'lon: {lon}, lat: {lat}' for (lon,lat) in per_simple_lonlat]
    ))

fig.add_trace(go.Scattermapbox(
        lat=[lat for (_,lat) in matched_micro_lonlat],
        lon=[lon for (lon,_) in matched_micro_lonlat],
        mode='lines',
        marker=go.scattermapbox.Marker(
            size=9
        ),
        text=[f'lon: {lon}, lat: {lat}' for (lon,lat) in matched_micro_lonlat]
    ))
# set render mode as html
fig.update_layout(mapbox_style="open-street-map", mapbox_center_lon=7.4232495, mapbox_center_lat=43.7397242, mapbox_zoom=14)
fig.show(renderer="browser")


In [ ]:
outputpathlen = tr.get_path_length_lonlat(lonlats=matched_micro_lonlat)
inputpathlen = tr.get_path_length_lonlat(per_simple_lonlat)
print(f"input path length: {inputpathlen}")
print(f"output path length: {outputpathlen}")
print(f"input/output: {inputpathlen/outputpathlen}")
mypygraph.get_path_length_osmid

In [ ]:
# #pickle the mapmatch_result and mapmatch_microresult
# import pickle
# with open('output/mapmatch_result.pickle', 'wb') as f:
#     pickle.dump(mapmatch_result, f)
# with open('output/mapmatch_microresult.pickle', 'wb') as f:
#     pickle.dump(mapmatch_microresult, f)
    

In [ ]:
flat_indices = np.array(mapmatch_microresult).flatten()
microlonlat = [(a,b) for (_,a,b) in map(mypygraph.get_node_weight, flat_indices)]

# plot the microresult on the map with plotly mapbox
fig = go.Figure(go.Scattermapbox(
        lat=[lat for (_,lat) in microlonlat],
        lon=[lon for (lon,_) in microlonlat],
        mode='lines',
        marker=go.scattermapbox.Marker(
            size=9
        ),
        text=[f'lon: {lon}, lat: {lat}' for (lon,lat) in microlonlat]
    ))
fig.update_layout(mapbox_style="open-street-map", mapbox_center_lon=7.4232495, mapbox_center_lat=43.7397242, mapbox_zoom=14)
fig.show(renderer="browser")

In [ ]:
some_candidates_lonlat = [(7.413833400000001, 43.727149000000004), (7.4131941, 43.727364200000004), (7.4133149000000005, 43.727352100000004), (7.413372900000001, 43.7276146), (7.412460500000001, 43.7278712), (7.4131992, 43.7274801), (7.4129911, 43.7275895), (7.412405000000001, 43.727939), (7.4132689, 43.727422700000005), (7.413555400000001, 43.727314), (7.4125347, 43.7278056), (7.4135207, 43.7276101), (7.4132674000000005, 43.7274534), (7.4134023000000004, 43.7274279), (7.4148388, 43.728591300000005), (7.4131967, 43.727426300000005), (7.4140771, 43.727924400000006), (7.4134108, 43.7274444), (7.413232900000001, 43.727405100000006), (7.4133525, 43.727477), (7.4180995, 43.7308279), (7.4134055000000005, 43.7274395), (7.4132483, 43.727421400000004), (7.4132997000000005, 43.7274216), (7.4124891, 43.727847100000005), (7.413246900000001, 43.727396000000006), (7.4131885, 43.727468300000005), (7.412992200000001, 43.7275652), (7.413270300000001, 43.7274049)]

# plot them on the map with plotly mapbox
fig = go.Figure(go.Scattermapbox(
        lat=[lat for (_,lat) in some_candidates_lonlat],
        lon=[lon for (lon,_) in some_candidates_lonlat],
        mode='lines',
        marker=go.scattermapbox.Marker(
            size=9
        ),
        text=[f'lon: {lon}, lat: {lat}' for (lon,lat) in some_candidates_lonlat]
    ))
fig.update_layout(mapbox_style="open-street-map", mapbox_center_lon=7.4232495, mapbox_center_lat=43.7397242, mapbox_zoom=14)
fig.show(renderer="browser")